#imports



In [ ]:
from tensorflow import keras
from functools import partial
from tensorflow.python.client import device_lib
import numpy as np

#Check current GPU

In [ ]:
print(device_lib.list_local_devices())

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 1079741337473580621
xla_global_id: -1
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 11321147392
locality {
  bus_id: 1
  links {
  }
}
incarnation: 3150033272920345238
physical_device_desc: "device: 0, name: Tesla K80, pci bus id: 0000:00:04.0, compute capability: 3.7"
xla_global_id: 416903419
]


#Fetch Dataset


In [ ]:
fashion_mnist = keras.datasets.fashion_mnist

(X_train_full, y_train_full), (X_test, y_test) = fashion_mnist.load_data()

X_valid, X_train = X_train_full[:5000] / 255.0, X_train_full[5000:] / 255.0
y_valid, y_train = y_train_full[:5000], y_train_full[5000:]

class_names = ["T-shirt/top", "Trouser", "Pullover", "Dress", "Coat", "Sandal", "Shirt", "Sneaker", "Bag", "Ankle boot"]

output_labels = len(class_names)
input_height = 28
input_width = 28

4431872/4422102 [==============================] - 0s 0us/step


In [ ]:
X_train.shape

(55000, 28, 28)

#RNN


In [ ]:
hidden_nodes = int(2*input_width*output_labels/3)

modelRNN = keras.models.Sequential([
  keras.layers.LSTM(hidden_nodes, return_sequences=False, input_shape=(input_width, input_height)),
  keras.layers.Dropout(0.2),
  keras.layers.Dense(units=output_labels, activation="softmax"),
])

modelRNN.compile(loss="sparse_categorical_crossentropy",optimizer="adam",metrics=["accuracy"])

In [ ]:
historyRNN = modelRNN.fit(X_train, y_train, epochs=10, validation_data=(X_valid, y_valid))

Epoch 1/10
1719/1719 [==============================] - 28s 13ms/step - loss: 0.5963 - accuracy: 0.7794 - val_loss: 0.4449 - val_accuracy: 0.8364
Epoch 2/10
1719/1719 [==============================] - 22s 13ms/step - loss: 0.4180 - accuracy: 0.8469 - val_loss: 0.3643 - val_accuracy: 0.8638
Epoch 3/10
1719/1719 [==============================] - 23s 13ms/step - loss: 0.3668 - accuracy: 0.8648 - val_loss: 0.3484 - val_accuracy: 0.8666
Epoch 4/10
1719/1719 [==============================] - 23s 13ms/step - loss: 0.3344 - accuracy: 0.8763 - val_loss: 0.3276 - val_accuracy: 0.8790
Epoch 5/10
1719/1719 [==============================] - 22s 13ms/step - loss: 0.3127 - accuracy: 0.8836 - val_loss: 0.2989 - val_accuracy: 0.8878
Epoch 6/10
1719/1719 [==============================] - 22s 13ms/step - loss: 0.2900 - accuracy: 0.8917 - val_loss: 0.2957 - val_accuracy: 0.8896
Epoch 7/10
1719/1719 [==============================] - 22s 13ms/step - loss: 0.2757 - accuracy: 0.8970 - val_loss: 0.2746 -

#2 Layer LSTM

In [ ]:
modelRNN2 = keras.models.Sequential([
  keras.layers.LSTM(input_height*input_width, return_sequences=True, input_shape=(input_width, input_height)),
  keras.layers.Dropout(0.2),
  keras.layers.LSTM(hidden_nodes, return_sequences=False, input_shape=(input_width, input_height)),
  keras.layers.Dropout(0.2),
  keras.layers.Dense(units=output_labels, activation="softmax"),
])

modelRNN2.compile(loss="sparse_categorical_crossentropy",optimizer="adam",metrics=["accuracy"])

historyRNN2 = modelRNN2.fit(X_train, y_train, epochs=10, validation_data=(X_valid, y_valid))

Epoch 1/10
1719/1719 [==============================] - 68s 37ms/step - loss: 0.5858 - accuracy: 0.7836 - val_loss: 0.4488 - val_accuracy: 0.8342
Epoch 2/10
1719/1719 [==============================] - 63s 37ms/step - loss: 0.4011 - accuracy: 0.8517 - val_loss: 0.3531 - val_accuracy: 0.8670
Epoch 3/10
1719/1719 [==============================] - 63s 37ms/step - loss: 0.3494 - accuracy: 0.8704 - val_loss: 0.3300 - val_accuracy: 0.8786
Epoch 4/10
1719/1719 [==============================] - 64s 37ms/step - loss: 0.3097 - accuracy: 0.8839 - val_loss: 0.2922 - val_accuracy: 0.8940
Epoch 5/10
1719/1719 [==============================] - 63s 37ms/step - loss: 0.2859 - accuracy: 0.8914 - val_loss: 0.2888 - val_accuracy: 0.8956
Epoch 6/10
1719/1719 [==============================] - 63s 37ms/step - loss: 0.2619 - accuracy: 0.9021 - val_loss: 0.2693 - val_accuracy: 0.8980
Epoch 7/10
1719/1719 [==============================] - 63s 37ms/step - loss: 0.2429 - accuracy: 0.9090 - val_loss: 0.2623 -

#CNN

In [ ]:
DefaultConv2D = partial(keras.layers.Conv2D,kernel_size=3, activation='relu', padding="SAME")

modelCNN = keras.models.Sequential([    
  DefaultConv2D(filters=64, kernel_size=7, input_shape=[input_width, input_height, 1]),    
  keras.layers.MaxPooling2D(pool_size=2),    
  DefaultConv2D(filters=128),    
  DefaultConv2D(filters=128),    
  keras.layers.MaxPooling2D(pool_size=2),    
  DefaultConv2D(filters=256),    
  DefaultConv2D(filters=256),    
  keras.layers.MaxPooling2D(pool_size=2),   
  keras.layers.Flatten(),    
  keras.layers.Dense(units=128, activation='relu'),    
  keras.layers.Dropout(0.5),   
  keras.layers.Dense(units=64, activation='relu'),    
  keras.layers.Dropout(0.5),    
  keras.layers.Dense(units=output_labels, activation='softmax'),
])

modelCNN.compile(loss="sparse_categorical_crossentropy",optimizer="adam",metrics=["accuracy"])

In [ ]:
history = modelCNN.fit(X_train, y_train, epochs=10, validation_data=(X_valid, y_valid))

Epoch 1/10
1719/1719 [==============================] - 54s 16ms/step - loss: 0.7821 - accuracy: 0.7197 - val_loss: 0.3767 - val_accuracy: 0.8582
Epoch 2/10
1719/1719 [==============================] - 28s 16ms/step - loss: 0.4549 - accuracy: 0.8430 - val_loss: 0.3174 - val_accuracy: 0.8864
Epoch 3/10
1719/1719 [==============================] - 27s 16ms/step - loss: 0.3866 - accuracy: 0.8674 - val_loss: 0.2895 - val_accuracy: 0.8942
Epoch 4/10
1719/1719 [==============================] - 28s 16ms/step - loss: 0.3409 - accuracy: 0.8826 - val_loss: 0.2950 - val_accuracy: 0.8972
Epoch 5/10
1719/1719 [==============================] - 27s 16ms/step - loss: 0.3096 - accuracy: 0.8949 - val_loss: 0.2720 - val_accuracy: 0.9018
Epoch 6/10
1719/1719 [==============================] - 27s 16ms/step - loss: 0.2873 - accuracy: 0.9011 - val_loss: 0.2526 - val_accuracy: 0.9094
Epoch 7/10
1719/1719 [==============================] - 28s 16ms/step - loss: 0.2729 - accuracy: 0.9071 - val_loss: 0.2577 -